实例59 (https://zhuanlan.zhihu.com/p/150110742) 探讨了批量转换`doc`文件为`docx`文件，并批量整理`docx`文件内容及文件名的“标记”词。现在遇到一个问题，那就是有些word文件是加密的，程序在执行的时候就会报错而导致中断。有没有办法把这些加密的文件挑出来放入单独的文件夹，以便后续手动处理呢？
<br/>
<br/>Python本身是难以识别word文件的加密状态的，那我们可以考虑利用程序在处理有密码的文档的时候的报错信息来进行处理。word文件有两种密码，一种是“打开密码”，另一种是“修改密码”。对于前者，若不输入正确的密码，是无法打开文件查看内容的；对于后者，即使不知道密码，还是可以用只读模式打开文件，查看内容并另存为新的文件的。
![](images\password.png)

因此对于设置了“修改密码”的`doc`文件，并不影响用如下程序另存为`docx`文件。只是在另存之前，窗口会弹出如下对话框，我们只需要点击一下“只读”即可，程序就能正常运行下去。
![](images\read_only.png)

如果设置了“打开密码”，程序在运行的时候也会弹出对话框，让输入密码。如果输入正确的密码，程序也是可以运行的；如果不知道密码或输入错误密码，则程序会出现如下的“com_error”错误。如果不知道密码，可以加入`try..except..`语句，让程序在`except`语句中将这个文件移动到名字为“有密码文件”的文件夹，以便后续处理(这里测试用的文件，密码为"123456")。

In [ ]:
from win32com import client as wc #导入模块
file = r"C:\Users\dywei\python\python_excel\实例65_Python区分出加密的word文件\doc文件\\测试-有打开密码.doc"
word = wc.Dispatch("Word.Application") # 打开word应用程序
doc = word.Documents.Open(file) #打开word文件
doc.SaveAs("{}x".format(file), 12)#另存为后缀为".docx"的文件，其中参数12指docx文件
doc.Close() #关闭原来word文件
word.Quit()

com_error: (-2147352567, '发生意外。', (0, 'Microsoft Word', '命令失败', 'wdmain11.chm', 36966, -2146824090), None)

此处，先了解一下程序执行的“异常”。当Python程序在执行的时候出现错误，它就会创建一个异常对象（即我们常见的`Traceback`）。如果没有处理异常的代码，程序就会中断并显示`Traceback`；如果编写了对应代码，程序会按代码的要求继续运行。处理异常，就要用到`try-except`代码块，它可以让Python执行指定的操作，同时告诉Python发生异常时的应对之策。比如如下程序，我们增加了`try-except`代码块，`try`里面是正常运行时的代码，`except`中则是出现错误的时候的代码。正常的话，是要将`doc`文件另存为`docx`文件。如果`doc`文件被设置了打开密码，则按`try`里的程序执行就会出错，会运行不下去，那就会自动触发`except`中的代码。这里，就会将这个加密了的文件移动到一个名字为“有密码文件”的文件夹中，以便后续手动处理。这样就间接的实现了识别有密码文件，并将其移动到指定文件夹。

In [12]:
#增加异常处理语句
import shutil #用于操作文件和文件夹
from win32com import client as wc 
file = r"C:\Users\dywei\python\python_excel\实例65_Python区分出加密的word文件\doc文件\\测试-有打开密码.doc"
try:
    word = wc.Dispatch("Word.Application") # 打开word应用程序
    doc = word.Documents.Open(file) #打开word文件
    doc.SaveAs("{}x".format(file), 12)#另存为后缀为".docx"的文件，其中参数12指docx文件
    doc.Close() #关闭原来word文件
    word.Quit()
except:
    #将有密码的文件移动到专门的文件夹
    shutil.move(file, r"C:\Users\dywei\python\python_excel\实例65_Python区分出加密的word文件\有密码文件")
    print(f"已移动加密文件 {file}")

已移动加密文件 C:\Users\dywei\python\python_excel\实例65_Python区分出加密的word文件\doc文件\\测试-有打开密码.doc


以上是“另存文件”为`docx`格式的处理方式。下面看看`docx`文件也有密码的情况。先测试一下文件有“打开密码”的情况，可见会出现“PackageNotFoundError”的错误。

PackageNotFoundError: Package not found at 'docx文件\报告-有打开密码.docx'

In [13]:
#文件有“打开密码”的情况
import docx
doc = docx.Document('docx文件\\报告-有打开密码.docx')

PackageNotFoundError: Package not found at 'docx文件\报告-有打开密码.docx'

再看看文件有“修改密码”的情况。运行程序，一切正常。可见不但能打开，而且可以写入，更厉害的是还能覆盖原文件保存。可见有“修改密码”，对于文件的读写没有任何影响。

In [15]:
#文件有“修改密码”的情况
import docx
doc = docx.Document('docx文件\\报告-有修改密码.docx')
p = doc.paragraphs[0] 
pNew = p.insert_paragraph_before() 
run = pNew.add_run("测试")
doc.save("docx文件\\报告-有修改密码.docx")

这样，我们只需要把有“打开密码”的文件进行处理。同样，加入`try-except`语句块，能正常打开读取的就按需要操作；不能打开的就移动到指定文件夹。这样就能比较巧妙地将有密码的word文件区分出来了。

In [16]:
import docx
import shutil
file = "docx文件\\报告-有打开密码.docx"

try:
    doc = docx.Document(file)
    pass
except:
    shutil.move(file, '有密码文件')
    print(f"已移动加密文件 {file} 到指定文件夹")

已移动加密文件 docx文件\报告-有打开密码.docx 到指定文件夹
